In [1]:
import pandas as pd
import numpy as np
import spectral as sp
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter
import time
import ipdb
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
from sklearn.decomposition import PCA
import sklearn
from glob import glob

import torch
%matplotlib inline

In [2]:
IMAGE_SIZE = [224, 224] # feel free to change depending on dataset

# training config:
epochs = 500
batch_size = 32

#define paths
covid_path = '../data/chest/Chest_COVID'
noncovid_path = '../data/chest/Chest_NonCOVID'

# Use glob to grab images from path .jpg or jpeg
covid_files = glob(covid_path + '/*')
noncovid_files = glob(noncovid_path + '/*')

In [3]:
import cv2
image = cv2.imread(covid_files[1])
image = np.transpose(image, (2, 1, 0))
image.shape

(3, 224, 224)

In [4]:
# Preparing Labels
covid_labels = []
noncovid_labels = []

covid_images=[]
noncovid_images=[]

import cv2 

for i in range(len(covid_files)):
  image = cv2.imread(covid_files[i])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  image = np.transpose(image, (2, 1, 0))
  covid_images.append(image)
  covid_labels.append('Chest_COVID')

for i in range(len(noncovid_files)):
  image = cv2.imread(noncovid_files[i])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  image = np.transpose(image, (2, 1, 0))
  noncovid_images.append(image)
  noncovid_labels.append('Chest_NonCOVID')

In [5]:
# normalize to interval of [0,1]
covid_images = np.array(covid_images) / 255
noncovid_images = np.array(noncovid_images) / 255

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

# split into training and testing
covid_x_train, covid_x_test, covid_y_train, covid_y_test = train_test_split(
    covid_images, covid_labels, test_size=0.2)
noncovid_x_train, noncovid_x_test, noncovid_y_train, noncovid_y_test = train_test_split(
    noncovid_images, noncovid_labels, test_size=0.2)


X_train = np.concatenate((noncovid_x_train, covid_x_train), axis=0)
X_test = np.concatenate((noncovid_x_test, covid_x_test), axis=0)
y_train = np.concatenate((noncovid_y_train, covid_y_train), axis=0)
y_test = np.concatenate((noncovid_y_test, covid_y_test), axis=0)

# make labels into categories - either 0 or 1
y_train = LabelBinarizer().fit_transform(y_train)
y_train = to_categorical(y_train)

y_test = LabelBinarizer().fit_transform(y_test)
y_test = to_categorical(y_test)

In [7]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 3, padding=1) # (N, 3, 224, 224) -> (N,  128, 224, 224)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)# (N, 128, 224, 224) -> (N,  256, 224, 224)
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1)# (N, 256, 224, 224) -> (N,  512, 224, 224)
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)# (N, 512, 224, 224) -> (N,  512, 224, 224)
        self.pool = nn.MaxPool2d(2, 2) # (N, 512, 224, 224) -> (N,  512, 112, 112)
        self.fc1 = nn.Linear(200704, 512)
        self.fc2 = nn.Linear(512, 1 )
#         self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
#         x = self.fc3(x)
        return x

device = torch.device("cuda")
net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [8]:
device

device(type='cuda')

In [9]:
X_train.shape

(752, 3, 224, 224)

In [10]:
Xt_train=torch.from_numpy(X_train)
yt_train = torch.from_numpy(y_train)

In [11]:
Xt_train.shape

torch.Size([752, 3, 224, 224])

In [12]:
for epoch in range(2):
    Xt_train= Xt_train.float()
    Xt_train = Xt_train.to(device)
    yt_train = yt_train.to(device)
    optimizer.zero_grad()
    pred = net(Xt_train)
    loss = criterion(pred, yt_train)
    loss.backward()
    optimizer.step()
    print(f'Step {step} loss {loss}')

RuntimeError: mat1 dim 1 must match mat2 dim 0

In [ ]:
print(net)

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [ ]:
! torch.cuda.empty_cache()